In [3]:
from pprint import pprint
import requests
import shadow_useragent
from bs4 import BeautifulSoup
import time
import random

In [12]:
ua = shadow_useragent.ShadowUserAgent()

my_user_agent = ua


In [13]:
search_url = 'https://fr.shopping.rakuten.com/search/harry+potter#nav=Livres'

headers = {
    
   'User-Agent': '{}'.format(my_user_agent),
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
}

session = requests.Session()
html = session.get(search_url, headers=headers)
#html = requests.get(search_url, headers=headers)
soup = BeautifulSoup(html.content, "html.parser")
html

<Response [403]>

In [ ]:
# Code du résultat
raw_grid = soup.select('.grid_container[data-qa=productListing] .grid_row')
raw_link = raw_grid[0].select('.panelCta_layoutBtn_ve9 a')

In [ ]:
# Lien des livres trouvés
url_books = []
for a in raw_link:
    if a['href'].find('[') == -1:
        url_books.append('https://fr.shopping.rakuten.com' + a['href'])
url_books

In [16]:
# infos de chacun de ces livres
info_books = []
info_book = {
    'titre': None,
    'image': None,
    'auteur': None,
    'prix': None,
    'note': None,
    'nbre_cmt': None,
    'best_seller': None,
    'liens': None,
    'id_search_word': None,
}
for url in url_books:
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Nom
    title = soup.select('#productName')[0].string.strip()
    info_book['titre'] = title
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Image
    img = soup.select('.prdMainPhotoCtn a')['href']
    info_book['nbre_cmt'] = img
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Auteur
    autor = soup.select('#prdTitleHead .prdTitleBrand')[0].string
    autor = autor.replace("-", "").strip()
    info_book['auteur'] = autor
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Prix
    if soup.select('#prdRightCol .buyBoxBlock[data-qa="buyBoxBlock"] .price'):
        price = soup.select('#prdRightCol .buyBoxBlock[data-qa="buyBoxBlock"] .price')[0].string
    elif soup.select('#prdRightCol #prdListingV2[data-qa="advertListingBlock"] .price'):
        price = soup.select('#prdRightCol #prdListingV2[data-qa="advertListingBlock"] .price')[0].string
    else:
        price = 'erreur'
    info_book['prix'] = price
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Note
    rating = soup.select('.ratingInfos .starRating .value')[0].string
    rating = rating.replace("_", ",")
    rating += " sur 5 étoiles" 
    info_book['note'] = rating
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Nombre de commentaires
    comments = soup.select('.ratingInfos .nb_reviews')[0].string
    info_book['nbre_cmt'] = comments
    time.sleep(round(random.uniform(1, 2), 1))
    
    ## Best seller ?
    if url == url_books[0]:
        best_seller = True
    else:
        best_seller = False
    info_book['best_seller'] = best_seller
    
    ## Lien
    info_book['liens'] = url
    
    ## id_search_word
    info_book['id_search_word'] = search_url.split("search/")[1]
    
    print(info_book)
    info_books.append(info_book)
    
info_books

[]